In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey
from haversine import haversine
from pprint import pprint

# needed to clear output below
from IPython.display import clear_output

In [3]:
fire_data = pd.read_csv("Fire.csv")
fire_data.head(1)

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000.0,1.01,1.23


In [4]:
dallas_coords_df = pd.read_csv("data/Dallas_coords_grid.csv")
dallas_coords_df.head(1)

,Lat,Lng,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,33.01025,-96.481611,2.21,2.76


In [9]:
fire_stations_df = pd.read_csv("data/fire_stations.csv")
fire_stations_df.head()

,Address,Lat,Lng
0,1901 Irving Blvd.,32.794005,-96.827024
1,4211 Northhaven Road,32.902599,-96.837838
2,500 N. Malcolm X Blvd.,32.787163,-96.783337
3,816 South Akard Street,32.774245,-96.796156
4,2039 St. Augustine Drive,32.675624,-96.657310


### Store distance to two nearest fire stations from each fire event.

In [15]:
station_dist_1 = []
station_dist_2 = []

for index, row in fire_data.iterrows():
    

    lat = row['Lat']
    lng = row['Lng']
    dist_list = []

    for index1, row1 in fire_stations_df.iterrows():
        lat1 = row1['Lat']
        lng1 = row1['Lng']
        dist = haversine(lat1,lng1,lat,lng,'mi')
        dist_list.append(dist)
    dist_list.sort()
    station_dist_1.append(dist_list[0])
    station_dist_2.append(dist_list[1])

fire_data['Dist to 1st Station (mi)'] = station_dist_1
fire_data['Dist to 2nd Station (mi)'] = station_dist_2
fire_data.to_csv('Fire.csv')

621

### Distance to nearest station for entire city of Dallas, regardless of fire event.

In [16]:
station_dist_1 = []
station_dist_2 = []

for index, row in fire_data.iterrows():
    

    lat = row['Lat']
    lng = row['Lng']
    dist_list = []

    for index1, row1 in dallas_coords_df.iterrows():
        lat1 = row1['Lat']
        lng1 = row1['Lng']
        dist = haversine(lat1,lng1,lat,lng,'mi')
        dist_list.append(dist)
    dist_list.sort()
    station_dist_1.append(dist_list[0])
    station_dist_2.append(dist_list[1])
fire_data['Dist to 1st Station (mi)'] = station_dist_1
fire_data['Dist to 2nd Station (mi)'] = station_dist_2
fire_data.to_csv('data/Dallas_coords_grid.csv')